# K-Nearest Neighbors (KNN) Algorithm

In this discussion, we will learn about the **K-Nearest Neighbor (KNN)** algorithm in Machine Learning. KNN is one of the simplest algorithms that can solve both **classification** and **regression** problems.

---

## 1. Classification with KNN

Consider a dataset with features \(F_1, F_2\) and output \(y\):

- \(y\) can be **binary** (0 or 1) or **multi-class**.
- \(F_1, F_2\) are the feature values.
- Example dataset:

| F1  | F2  | y |
|-----|-----|---|
| 1.2 | 3.4 | 0 |
| 2.1 | 1.8 | 1 |
| 0.5 | 2.2 | 0 |

We want to predict the category of a **new test point** using KNN.

### Steps:

1. **Initialize k value**  
   - The **k** value is a hyperparameter representing the number of nearest neighbors.
   - Typical values: \(k = 1, 3, 5, 10, \dots\)
   - Select k using **hyperparameter tuning** to maximize model accuracy.

2. **Find k nearest neighbors**  
   - Compute the distance between the **test point** and all **training points**.
   - Select the top k points with the smallest distances.

3. **Vote for category**  
   - Count how many neighbors belong to each category.
   - Assign the test point to the category with the **majority vote**.

Example:

- \(k = 5\)  
- Neighbor categories: [0, 0, 1, 1, 1]  
- Majority: 1 → Predicted category = 1

---

### Distance Metrics

To find nearest neighbors, we need a **distance metric**:

#### 1. Euclidean Distance
Distance between two points \((x_1, y_1)\) and \((x_2, y_2)\):

$$
d = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}
$$

For n-dimensional points:

$$
d = \sqrt{\sum_{i=1}^{n} (x_{2i} - x_{1i})^2}
$$

#### 2. Manhattan Distance
Distance along axes (like a grid):

$$
d = |x_2 - x_1| + |y_2 - y_1|
$$

- **Use-case**: City blocks or grid-based paths.
- Decide between Euclidean or Manhattan based on problem and accuracy.

---

## 2. Regression with KNN

For regression, the steps are the same as classification:

1. Find the **k nearest neighbors** of the test point.
2. Compute the **average** of their target values for prediction:

$$
y_{\text{pred}} = \frac{1}{k} \sum_{i=1}^{k} y_i
$$

- Optionally, use **median** instead of mean to reduce outlier impact.

Example: Predict house price based on house size:

- Training data: \([50, 60, 70, 80, 90]\) → prices \([150, 180, 210, 240, 270]\)  
- k = 3 nearest neighbors for size = 75 → prices = [180, 210, 240]  
- Predicted price = \((180 + 210 + 240)/3 = 210\)

---

## 3. KNN Variants for Optimization

- **Problem**: Naive KNN computes distance from the test point to all training points → **O(n)** time complexity.
- **Solution**: Use tree-based structures to reduce computation:
  1. **k-d Tree**
  2. **Ball Tree**

- These structures partition the data space, reducing unnecessary distance calculations.

---

## 4. Python Implementation Example

```python
# Import libraries
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
import numpy as np

# Sample dataset
X = np.array([[1.2, 3.4], [2.1, 1.8], [0.5, 2.2], [3.3, 0.4], [2.8, 3.5]])
y_class = np.array([0, 1, 0, 1, 1])  # Classification labels
y_reg = np.array([10, 20, 15, 25, 30])  # Regression values

# Classification
knn_class = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn_class.fit(X, y_class)
y_pred_class = knn_class.predict([[2.5, 2.0]])
print("Predicted class:", y_pred_class)

# Regression
knn_reg = KNeighborsRegressor(n_neighbors=3, metric='euclidean')
knn_reg.fit(X, y_reg)
y_pred_reg = knn_reg.predict([[2.5, 2.0]])
print("Predicted value:", y_pred_reg)


# KNN Variants: K-d Tree and Ball Tree

In the previous lecture, we discussed the **theoretical intuition of KNN**. In this lecture, we focus on **KNN variants** to optimize its time complexity.

---

## 1. Problem with Naive KNN

- In naive KNN, for a query point, we calculate the distance to **all training points**.
- Time complexity: **O(n)** for each query.
- For large datasets, this becomes computationally expensive.
- **Solution**: Use optimized structures like **K-d Tree** and **Ball Tree**.

---

## 2. K-d Tree (k-dimensional tree)

### Idea:
- K-d tree partitions points into a **binary tree structure**, reducing the number of distance calculations.
- Each split alternates between **x-axis (F1)** and **y-axis (F2)**.

### Construction Steps:

1. **Compute median of F1 (x-axis)**
   - Example F1 values: [2, 4, 5, 7, 8, 9]
   - Median ≈ 7 → use as the root split.
   
2. **Divide the points into two regions**
   - Left: F1 < 7
   - Right: F1 ≥ 7

3. **Next split uses F2 (y-axis)**
   - Compute median of F2 in each region.
   - Continue alternating x-axis and y-axis for further splits.

4. **Recursively split until all points are assigned**
   - Each node in the tree represents a point and a split.
   - Points are grouped into **regions**, so nearest neighbor search is limited to relevant regions.

### Nearest Neighbor Search:
- Traverse the tree to find the region containing the query point.
- Compute distances within the region.
- **Backtracking**: Move up the tree to check if closer neighbors exist in other regions.
- Example query:  
  Query point = (5, 7)  
  Nearest points found via tree traversal → reduces distance computations compared to brute-force.

---

## 3. Ball Tree

### Idea:
- Ball Tree clusters points hierarchically based on distance (like nested circles or "balls").
- Reduces backtracking compared to K-d Tree in high dimensions.

### Construction Steps:
1. Group nearby points into **clusters** (e.g., pairs or triples).
   - Example: Group1 = {2,3}, Group2 = {3,4}, Group3 = {5,6,7}, Group4 = {8,9}
2. Combine nearest clusters to form higher-level groups:
   - G5 = G1 + G2
   - G6 = G3 + G4
   - G7 = G5 + G6 (root of tree)
3. Each node stores points inside a "ball" (radius enclosing the group).

### Nearest Neighbor Search:
- Determine which **ball contains the query point**.
- Compute distances only within that ball.
- Move to next nearest balls if needed.
- Reduces distance computations significantly.

---

## 4. Comparison

| Feature           | K-d Tree                  | Ball Tree                   |
|------------------|--------------------------|----------------------------|
| Structure         | Binary tree              | Hierarchical clusters      |
| Split             | Axis-aligned (x, y alternately) | Based on distance (balls) |
| Backtracking      | Required                 | Less required              |
| Best for          | Low-dimensional data     | High-dimensional data      |
| Distance checks   | Reduced vs brute-force   | Further reduced            |

---

## 5. Python Example

```python
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

# Sample dataset
X = np.array([[2,3],[4,7],[5,4],[7,2],[8,1],[9,6]])
y = np.array([0, 0, 1, 1, 1, 0])

# KNN with Ball Tree
knn = KNeighborsClassifier(n_neighbors=3, algorithm='ball_tree')
knn.fit(X, y)

query_point = np.array([[5,7]])
prediction = knn.predict(query_point)
print("Predicted category:", prediction)
